In [3]:
# Store Data
# This part is similar to the first part
# of this process.
import os
import csv
import sys
import json
import glob
import pandas as pd

# The output files have a base of the string below.
BASE_FILE_NAME = "FILTERED_OUT"
MASTER_BASE_FILE_NAME = "OUT"

# These are the fields that are stored in the CSV
# file. Due to the overlap, I should define these constants
# elsewhere, in a shared file, but they will work here for now.
DOI = "doi"
TEXT = "text"
TITLE = "title"

# We're storing a subset of papers, so we use the number of
# that set in order to maintain that reference. I am saying
# not a lot with a lot of words.
def write_papers(number, papers):
    data_file_name = f"{BASE_FILE_NAME}_DATA_{number}.csv"
    dump_file_name = f"{BASE_FILE_NAME}_DUMP_{number}.csv"
    
    with open(data_file_name, 'w', newline='', encoding='utf-8-sig') as file:
        fieldnames = [TITLE, DOI, TEXT]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for paper in papers.values():
            writer.writerow(paper)
    
    with open(dump_file_name, 'w', newline='', encoding='utf-8') as file:
        json.dump(papers, file)

# This dictionary will store all the papers
# that we find through the various sources.
papers = {}
# To make sure that we're not doing anything
# stupid, there'll be a function that adds the papers.
# Use the function.
def add_paper(title, doi, text):
    number_papers = len(papers.keys())
    papers[number_papers] = {TITLE: title, DOI: doi, TEXT: text}
    number_papers += 1

# We read in a CSV file containing a list of papers to be filtered.
# The number indicates the number in the filename.
def read_papers(number):
    # This is taking too much work for me,
    # someone who is trying to get somewhere.
    # csv.field_size_limit(sys.maxsize//10)
    # papers = []
    file_name = f"{MASTER_BASE_FILE_NAME}_DATA_{number}.csv"
    # with open(file_name, 'r', encoding='utf-8') as file:
    #     csv_reader = csv.reader(file)
    #     header = next(csv_reader)
    #     for row in csv_reader:
    #         papers.append(row)
    df = pd.read_csv(file_name)
    return df
    # return papers

In [4]:
# There's no definite names for these patterns as I do not know what
# to call them. These patterns are used to extract possessive
# relationships from a sentence. I also could not find better names for
# the two variables below.
OWNER = "owner"
OWNED = "owned"

pattern_1 = [
    {
        "RIGHT_ID": OWNED,
        "RIGHT_ATTRS": {
            "POS": {
                "IN": ["NOUN"]
            }
        }
    },
    {
        "LEFT_ID": OWNED,
        "REL_OP": ">",
        "RIGHT_ID": OWNER,
        "RIGHT_ATTRS": {
            "DEP": "poss"
        }
    }
]

pattern_2 = [
     {
        "RIGHT_ID": OWNED,
        "RIGHT_ATTRS": {
            "POS": {
                "IN": ["NOUN"]
            }
        }
    },
    {
        "LEFT_ID": OWNED,
        "REL_OP": ">",
        "RIGHT_ID": "adp",
        "RIGHT_ATTRS": {
            "DEP": "prep",
            "POS": {
                "IN": ["ADP"]
            }
        }
    },
    {
        "LEFT_ID": "adp",
        "REL_OP": ">",
        "RIGHT_ID": OWNER,
        "RIGHT_ATTRS": {
            "DEP": "pobj",
            "POS": {
                "IN": ["NOUN"]
            }
        }
    }
]

pattern_3 = [
    {
        "RIGHT_ID": "verb",
        "RIGHT_ATTRS": {"POS": {"IN": ["VERB"]}}
    },
    {
        "LEFT_ID": "verb",
        "REL_OP": ">",
        "RIGHT_ID": OWNER,
        "RIGHT_ATTRS": {
            "DEP": "nsubj",
            "POS": {"IN": ["PRON"]}
        }
    },
    {
        "LEFT_ID": "verb",
        "REL_OP": ">",
        "RIGHT_ID": OWNED,
        "RIGHT_ATTRS": {
            "DEP": "dobj",
            "POS": {"IN": ["NOUN"]}
        }
    }
]

pattern_4 = [
    {
        "RIGHT_ID": "verb",
        "RIGHT_ATTRS": {"POS": {"IN": ["VERB"]}}
    },
    {
        "LEFT_ID": "verb",
        "REL_OP": ">",
        "RIGHT_ID": OWNED,
        "RIGHT_ATTRS": {
            "DEP": "nsubj",
            "POS": {"IN": ["NOUN"]}
        }
    },
    {
        "LEFT_ID": "verb",
        "REL_OP": ">",
        "RIGHT_ID": "adp",
        "RIGHT_ATTRS": {
            "DEP": "prep",
            "POS": {"IN": ["ADP"]}
        }
    },
    {
        "LEFT_ID": "adp",
        "REL_OP": ">",
        "RIGHT_ID": OWNER,
        "RIGHT_ATTRS": {
            "DEP": "pobj",
            "POS": {"IN": ["NOUN"]}
        }
    }
]

patterns = {
    "Pattern1": pattern_1,
    "Pattern2": pattern_2,
    "Pattern3": pattern_3,
    "Pattern4": pattern_4,
}

def dependency_matcher(sp_nlp):
    matcher = DependencyMatcher(sp_nlp.vocab)
    for pattern_id, pattern in patterns.items():
        matcher.add(pattern_id, [pattern])
    return matcher

In [5]:
import spacy
import stanza
import textacy
from fastcoref import FCoref
from taxonerd import TaxoNERD
from spacy.matcher import Matcher
from spacy.matcher import DependencyMatcher
# !pip install https://github.com/nleguillarme/taxonerd/releases/download/v1.5.4/en_ner_eco_md-1.1.0.tar.gz
!pip install https://github.com/nleguillarme/taxonerd/releases/download/v1.5.4/en_ner_eco_biobert-1.1.0.tar.gz
# !pip install https://github.com/nleguillarme/taxonerd/releases/download/v1.5.4/en_ner_eco_md_weak-1.1.0.tar.gz
# !pip install https://github.com/nleguillarme/taxonerd/releases/download/v1.5.4/en_ner_eco_biobert_weak-1.1.0.tar.gz

C:\Users\lbeln\anaconda3\envs\3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


     ---------------------------------------- 0.0/520.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/520.9 MB ? eta -:--:--
     -------------------------------------- 0.0/520.9 MB 131.3 kB/s eta 1:06:08
     -------------------------------------- 0.0/520.9 MB 131.3 kB/s eta 1:06:08
     -------------------------------------- 0.0/520.9 MB 196.9 kB/s eta 0:44:05
     -------------------------------------- 0.1/520.9 MB 233.8 kB/s eta 0:37:08
     -------------------------------------- 0.1/520.9 MB 245.8 kB/s eta 0:35:20
     -------------------------------------- 0.1/520.9 MB 448.2 kB/s eta 0:19:22
     -------------------------------------- 0.3/520.9 MB 912.8 kB/s eta 0:09:31
     ---------------------------------------- 1.2/520.9 MB 2.8 MB/s eta 0:03:08
     ---------------------------------------- 2.9/520.9 MB 6.1 MB/s eta 0:01:26
     ---------------------------------------- 5.4/520.9 MB 9.6 MB/s eta 0:00:54
      -------------------------------------- 7.4/520.9

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
sp_nlp = spacy.load("en_core_web_sm")
st_nlp = stanza.Pipeline(lang='en', processors='tokenize')
fcoref = FCoref(enable_progress_bar=False)
taxonerd = TaxoNERD()
tn_nlp = taxonerd.load(model="en_ner_eco_biobert")

2025-04-07 10:39:13 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
04/07/2025 10:39:13 - INFO - 	 Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-04-07 10:39:13 INFO: Downloaded file to C:\Users\lbeln\stanza_resources\resources.json
04/07/2025 10:39:13 - INFO - 	 Downloaded file to C:\Users\lbeln\stanza_resources\resources.json
2025-04-07 10:39:14 WARNING: Language en package default expects mwt, which has been added
04/07/2025 10:39:14 - WARNING - 	 Language en package default expects mwt, which has been added
2025-04-07 10:39:14 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |

04/07/2025 10:3

In [7]:
def index_to_token(sp_doc):
    index_to_token_map = {}
    for token in sp_doc:
        index_to_token_map[token.idx] = token
    return index_to_token_map

def index_to_cluster(fc_predictions):
    index_to_cluster_map = {}
    for prediction in fc_predictions:
        clusters = prediction.get_clusters(as_strings=False)
        for cluster in clusters:
            for token in cluster:
                index = token[0]
                index_to_cluster_map[index] = cluster
    return index_to_cluster_map

def index_to_chunk(sp_doc):
    index_to_chunk_map = {}
    for noun_chunk in sp_doc.noun_chunks:
        for token in noun_chunk:
            index_to_chunk_map[token.idx] = noun_chunk
    return index_to_chunk_map

def index_to_what(sp_nlp, sp_doc, what_matches):
    index_to_what_map = {}
    for match_id, token_ids in what_matches:
        pattern_id = sp_nlp.vocab.strings[match_id]
        owner = None
        owned = None
        for i in range(len(token_ids)):
            right_id = patterns[pattern_id][i]["RIGHT_ID"]
            if right_id == OWNER:
                owner = sp_doc[token_ids[i]]
            if right_id == OWNED:
                owned = sp_doc[token_ids[i]]
        if owner.idx not in index_to_what_map:
            index_to_what_map[owner.idx] = []
        index_to_what_map[owner.idx].append(owned)
        if owned.idx not in index_to_what_map:
            index_to_what_map[owned.idx] = []
        index_to_what_map[owned.idx].append(owned)

    return index_to_what_map

def species_indices(tn_doc):
    indices = []
    for species_span in tn_doc.ents:
        for species in species_span:
            indices.append(species.idx)
    return indices

def context(sp_doc, tokens, token_map, cluster_map, chunk_map, what_map):
    token_indices = [token.idx for token in tokens]
    what = []
    chunks = []
    clusters = []
    for token_index in token_indices:
        # Clusters
        if token_index in cluster_map:
            for cluster_token_index in cluster_map[token_index]:
                clusters.append(token_map[cluster_token_index[0]])
        # Chunks
        if token_index in chunk_map:
            for token in chunk_map[token_index]:
                chunks.append(token)
        # What
        if token_index in what_map:
            for token in what_map[token_index]:
                what.append(token)
    return ([*clusters, *chunks, *what], {"clusters": clusters, "chunks": chunks, "what": what})

def is_species(tokens, context, species_indices):
    for token in [*tokens, *context]:
        if token.idx in species_indices:
            print(f"Token '{token.text}' is a Species")
            return True
    return False

def is_relevant(text):
    sp_doc = sp_nlp(text)
    token_map = index_to_token(sp_doc)
    chunk_map = index_to_chunk(sp_doc)

    matcher = dependency_matcher(sp_nlp)
    matches = matcher(sp_doc)
    what_map = index_to_what(sp_nlp, sp_doc, matches)
    
    tn_doc = tn_nlp(text)
    species = species_indices(tn_doc)

    predictions = fcoref.predict(texts=[text])
    cluster_map = index_to_cluster(predictions)

    found_instance = False
    for sentence in sp_doc.sents:
        svo_triples = textacy.extract.subject_verb_object_triples(sp_doc)
        for svo_triple in svo_triples:
            sub_context = context(sp_doc, svo_triple.subject, token_map, cluster_map, chunk_map, what_map)[0]
            print(f"Sub. Context: {sub_context}")
            valid_sub = is_species(svo_triple.subject, sub_context, species)

            obj_context = context(sp_doc, svo_triple.object, token_map, cluster_map, chunk_map, what_map)[0]
            print(f"Obj. Context: {obj_context}")
            valid_obj = is_species(svo_triple.object, obj_context, species)

            if valid_sub and valid_obj:
                print(f"Sentence: {sentence}")
                print(svo_triple)
                print(sub_context)
                print(obj_context)
                found_instance = True
    return found_instance

In [ ]:
df = read_papers(6)
df.reset_index()
# SLOW
# for index, row in df.iterrows():
#     if not row["text"] or not is_relevant(row["text"]):
#         continue
#     add_paper(row["title"], row["doi"], row["text"])
# for text in df["text"]:
if is_relevant(df["text"][0]):
    print("Match Found")

04/07/2025 10:41:29 - INFO - 	 Tokenize 1 inputs...
Map: 100%|██████████| 1/1 [00:01<00:00,  1.57s/ examples]
04/07/2025 10:41:30 - INFO - 	 ***** Running Inference on 1 texts *****


Sub. Context: [these, cells]
Obj. Context: [homeostatic, functions, functions]
Sub. Context: [Cells, Cells]
Obj. Context: [considerable, diversity, plasticity]
Sub. Context: [mononuclear, phago-, mononuclear, phago-]
Obj. Context: [respond, respond]
Sub. Context: [macrophages, macrophages]
Obj. Context: [M1, M1, M1, M1, M1, M1, M1, M1, M1, classical, M1, activation, classical, M1, activation, alternative, M2, activation, alternative, M2, activation]
Sub. Context: [these, 
, states]
Obj. Context: [the, Th1, –, Th2, polarization, polarization]
Sub. Context: [M1, M1, M1, M1, M1, M1, M1, M1, M1, The, M1, phenotype, The, M1, phenotype]
Obj. Context: [the, expression, expression]
Sub. Context: [M2, They, M2, 
, macrophages, M2, 
, macrophages]
Obj. Context: [parasite, containment, parasite, containment, promotion, tissue, remodeling, tissue, remodeling, tumor, progression, tumor, progression, immunoregulatory, functions, immunoregulatory, functions, promotion, promotion]
Sub. Context: [M2, T